## Discussion 3:  Pandas

In this discussion, we will cover some common operations that you will use in Pandas, including `groupby`, aggregates, `filter`, and `apply`. 

For a more detailed version of the questions in this section, please see the [discussion worksheet](http://www.ds100.org/sp20/resources/assets/discussions/disc03.pdf).

The two cells bellow will download the relevant data and load it into two dataframes locally called `elections` and `babynames`.

In [150]:
import pandas as pd
import requests
import zipfile
from pathlib import Path

import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head(5)

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


In [139]:
elections = pd.read_csv("elections.csv")
elections.head(5)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789


### Elections Data

In this first sequence of questions, we'll analyze the elections data. These first three exercises correspond to questions 4a-4c on the discussion worksheet.

#### Question 4a

Using `groupby.agg` or one of the shorthand methods (`groupby.min`, `groupby.first`, etc.), create a `Series` best result that gives the highest percentage vote ever attained by each party. For example, best `result[‘Libertarian’]` should return 3.3. The order of your `Series` does not matter.

In [151]:
best_result_by_party = elections \
    .groupby('Party')['%'] \
    .agg('max')
best_result_by_party

Party
American                 21.554001
American Independent     13.571218
Anti-Masonic              7.821583
Anti-Monopoly             1.335838
Citizens                  0.270182
Communist                 0.261069
Constitution              0.152398
Constitutional Union     12.639283
Democratic               61.344703
Democratic-Republican    57.210122
Dixiecrat                 2.412304
Farmer–Labor              0.995804
Free Soil                10.138474
Green                     2.741176
Greenback                 3.352344
Independent              18.956298
Liberal Republican       44.071406
Libertarian               3.307714
National Democratic       0.969566
National Republican      43.796073
National Union           54.951512
Natural Law               0.118219
New Alliance              0.237804
Northern Democratic      29.522311
Populist                  8.645038
Progressive              27.457433
Prohibition               2.249468
Reform                    8.408844
Republican    

#### Question 4b

Again using `groupby.agg` or one of the its shorthand methods, create a `DataFrame` that gives the result for a party in its most recent year of participation, with Party as its index. For example `last_result.query("Party == ‘Whig’")` should give you a row showing that the Whigs last participated in an election in 1852 with Winfield Scott as their candidate, earning 44% of the vote. This might take more than one line of code. Write your answer below.

In [168]:
## SOLUTION 1: USING APPLY
def pick_max_of_party(frame):
    return frame[frame['Year'] == frame['Year'].max()]

last_year_by_party = elections \
    .groupby('Party') \
    .apply(pick_max_of_party)

## SOLUTION 2: USING AGG/FIRST

last_year_by_party = elections.sort_values('Year', ascending=False) \
    .groupby('Party') \
    .first() # or .agg('first')

### **Ask yourself what the difference between these two solutions might be?**

last_year_by_party

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,158271,loss,0.194862
American Independent,1976,Lester Maddox,170274,loss,0.209640
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Darrell Castle,203091,loss,0.149640
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2016,Hillary Clinton,65853514,loss,48.521539


#### Question 4c

Using filter, create a `DataFrame` of major party results since 1988 that includes all election results starting in 1988, but only includes each row if the Party it belongs to has earned at least 1% of the popular vote in ***any*** election since 1988. For example, in 1988, you should not include the "New Alliance" candidate since this party has not earned 1% of the vote since 1988. However, you should include the "Libertarian" candidate from 1988 who only earned 0.47% of the vote in 1988 because in 2016 the Libertarian candidate Gary Johnson had 3.3% of the vote.

In [149]:
major_party_results_since_1988 = elections[elections['Year'] >= 1988] \
    .groupby('Party') \
    .filter(lambda frame: frame['%'].max() >= 1.0)
major_party_results_since_1988

,Year,Candidate,Party,Popular vote,Result,%
135,1988,George H. W. Bush,Republican,48886597,win,53.518845
137,1988,Michael Dukakis,Democratic,41809074,loss,45.770691
138,1988,Ron Paul,Libertarian,431750,loss,0.472660
139,1992,Andre Marrou,Libertarian,290087,loss,0.278516
140,1992,Bill Clinton,Democratic,44909806,win,43.118485
142,1992,George H. W. Bush,Republican,39104550,loss,37.544784
143,1992,Ross Perot,Independent,19743821,loss,18.956298
144,1996,Bill Clinton,Democratic,47400125,win,49.296938
145,1996,Bob Dole,Republican,39197469,loss,40.766036
146,1996,Harry Browne,Libertarian,485759,loss,0.505198


### Baby Names Data

Now we'll turn our attention to the baby names dataset (the `babynames` `DataFrame` we loaded above). This section corresponds to exercises 4d-4e on the discusson worksheet.

#### Question 4d

Create a `Series` named `female_names_since_2000_count` which gives the total number of occurrences of each name for female babies born in California from the year 2000 or later. The index should be the name, and the value should be the total number of births. Your series should be ordered in decreasing order of count. For example, your first row should have the index “Emily” and the value 49605, because 49,605 Emilys have been born since the year 2000 in California.

In [146]:
female_names_since_2000_count = babynames[(babynames['Year'] >= 2000) & (babynames['Sex'] == 'F')] \
    .groupby('Name')['Count'] \
    .sum() \
    .sort_values(ascending=False) 
female_names_since_2000_count

Name
Emily       49605
Isabella    47447
Sophia      46113
Mia         38143
Emma        37622
            ...  
Brigida         5
Mansirat        5
Manmeet         5
Brilee          5
Meily           5
Name: Count, Length: 9090, dtype: int64

#### Question 4e

Using `groupby`, create a `Series` called `count_for_names_2018` listing all baby names from 2018 in decreasing order of popularity. The result should not be broken down by gender! If a name is used by both male and female babies, the number you provide should be the total across both genders. For example, `count_for_names_2018["Noah"]`
should be the number 2567 because in 2018 there were 2567 Noahs born (12 female and 2555 male).

In [142]:
count_for_names_2018 = babynames[babynames['Year'] == 2018] \
    .groupby('Name')['Count'] \
    .sum() \
    .sort_values(ascending=False)
count_for_names_2018

Name
Emma        2722
Noah        2567
Mia         2484
Olivia      2456
Liam        2405
            ... 
Naelani        5
Janiya         5
Kuzey          5
Cyan           5
Tonatiuh       5
Name: Count, Length: 6191, dtype: int64